In [5]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [6]:
# 查看当前kernerl下的package
!pip list --format=columns

In [7]:
# 显示cell运行时长
%load_ext klab-autotime

In [9]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn import metrics
from sklearn.model_selection import KFold
from fastText import train_supervised

In [10]:
path="/home/kesci/input/bytedance/first-round/"
train = pd.read_csv(path+'train.csv', header=None, chunksize=5000000)
test = pd.read_csv(path+'test.csv', header=None)
pre_df = test[[0, 2]]

In [3]:
# ct = CountVectorizer()
# d = train[1] + ' ' +train[3]
# x_train = ct.fit_transform(d)

# d = test[1] + ' ' +test[3]
# x_test = ct.transform(d)

# del d

In [11]:
#lr+CountVectorizer

# def getfTitleOhot(df):
#     a = np.zeros((df.shape[0], 20), dtype=np.int8)
#     for i in range(len(d)):
#         b = d[i].split(' ')
#         for j in b:
#             a[i][int(j)] = int(j)
            
#     return a

kf_n = 2
kf = KFold(n_splits=kf_n, shuffle=True, random_state=2)
pre = 0

# model = SGDClassifier(penalty='l1', random_state = 2, shuffle = True, loss='log', alpha=0.01)
model = LogisticRegression(penalty='l1', C=0.14, n_jobs=-1)
x = 1
for df in train:
    
    ct = CountVectorizer(ngram_range=(1, 2), min_df=50)
    # ct = CountVectorizer(ngram_range=(1, 2))
    # ct = TfidfVectorizer(sublinear_tf=True,ngram_range=(1, 2), max_df=0.5, analyzer='word', norm='l1')
    
    print('第%d次训练开始'%x)
    
    #训练集
    d = df[1] + ' ' +df[3]
    y_train = df[4].copy()
    x_train = ct.fit_transform(d)
    
    # title_oneh = np.array(pd.get_dummies(df[2]))
    # x_train = np.hstack((x_train, title_oneh))
    # del title_oneh
        
    print('训练集处理完成')

    #测试集
    d = test[1] + ' ' +test[3]
    x_test = ct.transform(d)
    
    # title_oneh = np.array(pd.get_dummies(test[2]))
    # x_train = np.hstack((x_train, title_oneh))
    # del title_oneh
    
    print('测试集处理完成')
    
    del d

        
    for x_tr, x_vail in kf.split(x_train):
        x_t1 = x_train[x_tr]
        y_t1 = y_train.iloc[x_tr]
        
        x_t2 = x_train[x_vail]
        y_t2 = y_train.iloc[x_vail]
        
        model.fit(x_t1, y_t1)
        print(metrics.roc_auc_score(y_t2, model.predict_proba(x_t2)[:, 1]))
        pre += model.predict_proba(x_test)[:, 1]
        
    # model.fit(x_train, y_train)
    # pre += model.predict_proba(x_test)[:, 1]
        
    del x_train
    del x_test
    del ct
    
    print('第%d次训练结束'%x)

    x += 1

    
# pre1 = pre/(kf_n*(x-1))
pre1 = pre/(x-1)

pre_df = pre_df.rename(columns={0:'query_id', 2:'query_title_id'})
pre_df['prediction'] = pre1
pre_df.to_csv('lr_ct_500wEoch_saga.csv', index=False, header=False)

'''
5折交叉验证+500w一次,验证得分0.574
2折交叉验证+1000w一次,验证得分0.568
'''

In [11]:
del train

train = pd.read_csv(path+'train.csv', header=None, chunksize=100)

In [12]:
# 增量训练

def getfeatrue(df):
    # a = np.zeros((df.shape[0], 2000000), dtype=np.int8)
    d = list(df[1] + ' ' +df[3])
    for i in range(len(d)):
        a = np.zeros(1000000, dtype=np.int8)
        for j in d[i].split(' '):
            try:
                a[int(j)] += 1
            except:
                pass
        if i < 1:
            v = a
        else:
            v = np.vstack((v, a))

    return v
            
        
# x_test = getfeatrue(test)
# print('测试集处理完成')

kf = KFold(n_splits=5, shuffle=True, random_state=2019)
pre = 0

model = SGDClassifier(penalty='l1', random_state = 2, shuffle = True, loss='log', alpha=0.01, n_jobs=-1)
# model = LogisticRegression(penalty='l1', C=0.21)
x = 1
# ct = CountVectorizer(ngram_range=(1, 2))
for df in train:
    
    x_train = getfeatrue(df)
    y_train = df[4].copy()
    
    del df
    
    print('第%d训练集处理完成'%x)


    for x_tr, x_vail in kf.split(x_train):
        x_t1 = x_train[x_tr]
        y_t1 = y_train.iloc[x_tr]
        
        x_t2 = x_train[x_vail]
        y_t2 = y_train.iloc[x_vail]
        
        model.partial_fit(x_t1, y_t1, classes=[0, 1])
        try:
            print(metrics.roc_auc_score(y_t2, model.predict_proba(x_t2)[:, 1]))
        
        # pre += model.predict_proba(x_test)[:, 1]

    x += 1
    # if x>10:
    #     break
    
    
# pre1 = pre/(2*(x-1))
# pre_all = model.predict_proba(x_test)[:, 1]

# pre_df = pre_df.rename(columns={0:'query_id', 2:'query_title_id'})
# pre_df['prediction'] = pre1
# pre_df.to_csv('lr_ct_增量训练.csv', index=False, header=False)

第1训练集处理完成
0.34374999999999994
0.6483516483516484
0.6111111111111112
1.0
0.9880952380952381
第2训练集处理完成
0.546875
0.7450980392156863
0.609375
0.4166666666666667
0.75
第3训练集处理完成
0.3958333333333333
0.5714285714285715
0.6190476190476191
0.8733333333333334
0.859375
第4训练集处理完成
0.6
0.7380952380952381
0.765625
0.703125
第5训练集处理完成
0.9607843137254902
0.6796875
0.9166666666666665
0.8383838383838383
0.7600000000000001
第6训练集处理完成
0.4945054945054945
0.21052631578947367
0.65625
0.8400000000000001
0.484375
第7训练集处理完成
0.4642857142857143
0.5733333333333334
0.96875
0.625
0.5294117647058824
第8训练集处理完成
0.703125
0.3214285714285714
0.37362637362637363
0.6372549019607843
0.6274509803921569
第9训练集处理完成
0.49999999999999994
0.9444444444444444
0.7777777777777779
0.696969696969697
0.6666666666666666
第10训练集处理完成
0.5364583333333333
0.7023809523809523
0.7777777777777777
0.6666666666666667
0.9375
第11训练集处理完成
0.640625
0.7466666666666667
0.8823529411764706
0.8725490196078431
0.4747474747474747
第12训练集处理完成
0.71875
0.7321428571428572
0

KeyboardInterrupt: 

In [23]:
def getfeatruetest(df):
    d = list(df[1] + ' ' +df[3])
    for i in range(len(d)):
        a = np.zeros(1000000, dtype=np.int8)
        for j in d[i].split(' '):
            try:
                a[int(j)] += 1
            except:
                pass
        # if i < 1:
        #     v = a
        # else:
        #     v = np.vstack((v, a))

    return a
    
pre_all = []
for i in range(len(test)):
    # print(test.iloc[i, :])
    x_test = getfeatruetest(test.iloc[i, :])
    x_test = np.atleast_2d(x_test)

    pre_all.append(model.predict_proba(x_test)[:, 1])
    if i%1000 == 0:
        print(model.predict_proba(x_test)[:, 1])


pre_df = pre_df.rename(columns={0:'query_id', 2:'query_title_id'})
pre_df['prediction'] = pre_all
pre_df.to_csv('lr_ct_增量训练_100W.csv', index=False, header=False)

[0.25307733]
[0.25307733]


KeyboardInterrupt: 

[array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.25307733]),
 array([0.

In [ ]:
# # hashing
# train = pd.read_csv(path+'train.csv', header=None, chunksize=5000000)

# kf = KFold(n_splits=5, shuffle=True, random_state=2019)
# pre = 0

# model = SGDClassifier(penalty='l1', random_state = 2, shuffle = True, loss='log', alpha=0.01)
# # model = LogisticRegression(penalty='l1', C=0.21)
# x = 1
# for df in train:
    
#     hv = HashingVectorizer(ngram_range=(1, 2), n_features=200)
#     # ct = TfidfVectorizer(sublinear_tf=True,ngram_range=(1, 2), max_df=0.5, analyzer='word', norm='l1')
#     # ct = TfidfVectorizer()
#     d = df[1] + ' ' +df[3]
#     x_train = hv.fit_transform(d)
#     y_train = df[4].copy()
    
#     del df
    
#     d = test[1] + ' ' +test[3]
#     x_test = hv.transform(d)
    
#     del d

        
#     for x_tr, x_vail in kf.split(x_train):
#         x_t1 = x_train[x_tr]
#         y_t1 = y_train.iloc[x_tr]
        
#         x_t2 = x_train[x_vail]
#         y_t2 = y_train.iloc[x_vail]
        
#         model.fit(x_t1, y_t1)
#         print(metrics.roc_auc_score(y_t2, model.predict_proba(x_t2)[:, 1]))
#         pre += model.predict_proba(x_test)[:, 1]

#     x += 1
    
# pre1 = pre/(5*(x-1))
# pre_df_hash = pre_df.rename(columns={0:'query_id', 2:'query_title_id'})
# pre_df_hash['prediction'] = pre1
# pre_df_hash.to_csv('lr_ct_hashing.csv', index=False, header=False)

# del train

# train = pd.read_csv(path+'train.csv', header=None, chunksize=5000000)

In [ ]:
# pre_df = pre_df.rename(columns={0:'query_id', 2:'query_title_id'})
# pre_df['prediction'] = pre1
# pre_df.to_csv('lr_ct_增量训练.csv', index=False, header=False)

In [1]:
# pre_df.to_csv('lr_ct.csv', index=False, header=False)
!./kesci_submit -token 5ebea4ff5f5fa8e0 -file lr_ct_500wEach.csv